In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GRU, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import itertools
import os
import random

# MPA algoritması
def mpa_optimizer(hyperparams, fitness_function, num_iterations=50):
    num_agents = len(hyperparams)
    best_solution = None
    best_fitness = float('inf')

    # Rastgele başlangıç populasyonu
    positions = np.random.uniform(low=0, high=1, size=(num_agents, len(hyperparams[0])))

    for iteration in range(num_iterations):
        for i in range(num_agents):
            # Rastgele bir yeni pozisyon seç
            new_position = np.random.uniform(low=0, high=1, size=(1, len(hyperparams[0])))

            # Yeni pozisyonun fitness değeri
            fitness = fitness_function(new_position[0])
            if fitness < best_fitness:
                best_fitness = fitness
                best_solution = new_position

    return best_solution[0]

# CSTOA algoritması
def cstoa_optimizer(init_population, fitness_function, num_iterations=50):
    num_agents = len(init_population)
    best_solution = None
    best_fitness = float('inf')

    # Kaotik haritalama ile populasyonu başlat
    positions = init_population * np.random.uniform(size=(num_agents, len(init_population[0])))

    for iteration in range(num_iterations):
        for i in range(num_agents):
            # Yeni pozisyonu güncelleme
            new_position = positions[i] + np.random.randn(*positions[i].shape)
            fitness = fitness_function(new_position)

            # En iyi çözüm güncelle
            if fitness < best_fitness:
                best_fitness = fitness
                best_solution = new_position

    return best_solution

# CLAHE ile ön işleme
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Hata: {image_path} konumundaki görüntü okunamadı.")
        return None
    image = cv2.resize(image, (224, 224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image = clahe.apply(image)
    image = image / 255.0
    image = np.stack([image, image, image], axis=-1)
    return image

# Veriyi yükleme ve ön işleme
def load_data(base_dir):
    classes = ['Dense', 'Medium_Dense', 'Sparse', 'None']
    image_paths = []
    labels = []

    for cls in classes:
        class_dir = os.path.join(base_dir, cls)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            processed_image = preprocess_image(img_path)
            if processed_image is not None:
                image_paths.append(processed_image)
                labels.append(cls)

    data = np.array(image_paths)
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    return data, labels

# Model oluşturma (Inception v3 + GRU)
def build_model(hyperparams):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    model = Sequential([
        base_model,
        Flatten(),
        Dense(int(hyperparams[0]), activation='relu'),
        Reshape((1, int(hyperparams[0]))),
        GRU(max(1, int(hyperparams[1])), return_sequences=True),
        GRU(max(1, int(hyperparams[2]))),
        Dense(4, activation='softmax')
    ])

    learning_rate = hyperparams[3]
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Hiperparametre optimizasyonu
def fitness_function(hyperparams):
    model = build_model(hyperparams)
    model, _ = train_model(model, data, labels, epochs=3)
    loss, accuracy = model.evaluate(data, labels)
    return 1 - accuracy

# Modeli eğitme
def train_model(model, data, labels, epochs=100, callbacks=None):
    X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_val, y_val), callbacks=callbacks)
    return model, history

# Modeli değerlendirme
def evaluate_model(model, data, labels):
    predictions = model.predict(data)
    predictions = np.argmax(predictions, axis=1)
    report = classification_report(labels, predictions, target_names=['Dense', 'Medium Dense', 'Sparse', 'None'])
    print(report)

# Karışıklık matrisi çizimi
def plot_confusion_matrix(y_true, y_pred, classes, title='Confusion Matrix'):
    cm = confusion_matrix(y_true, y_pred)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

# Doğruluk ve kayıp grafikleri çizimi
def plot_accuracy_and_loss(history):
    plt.figure()
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# PR eğrisi çizimi
def plot_pr_curve(y_true, y_scores, class_names):
    for i, class_name in enumerate(class_names):
        precision, recall, _ = precision_recall_curve(y_true == i, y_scores[:, i])
        plt.plot(recall, precision, lw=2, label=f'{class_name}')

    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve")
    plt.legend(loc="best")
    plt.show()

# ROC eğrisi çizimi
def plot_roc_curve(y_true, y_scores, class_names):
    for i, class_name in enumerate(class_names):
        fpr, tpr, _ = roc_curve(y_true == i, y_scores[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'{class_name} (AUC = {roc_auc:.2f})')

    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="best")
    plt.show()

# Ana program
if __name__ == '__main__':
    base_dir = '/content/drive/MyDrive/Colab Notebooks/dataset/'
    data, labels = load_data(base_dir)

    # Rastgele hiperparametrelerle başlangıç populasyonu oluştur
    init_population = np.random.uniform(low=[32, 1, 1, 0.0001], high=[256, 128, 64, 0.01], size=(10, 4))

    # Hiperparametreleri MPA ve CSTOA ile optimize et
    best_hyperparams_cstoa = cstoa_optimizer(init_population, fitness_function)

    # En iyi hiperparametrelerle model oluştur ve eğit
    model = build_model(best_hyperparams_cstoa)

    # Early stopping callback'ini oluşturun
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Modeli farklı epoch değerleri için eğit
    epoch_values = [500, 1000, 1500]
    for epochs in epoch_values:
        print(f'{epochs} epoch ile eğitim yapılıyor...')
        model, history = train_model(model, data, labels, epochs=epochs, callbacks=[early_stopping])

        # Modeli değerlendir ve metrikleri görüntüle
        evaluate_model(model, data, labels)

        # Tahminleri al ve sonuçları çiz
        y_pred = model.predict(data)
        y_pred_classes = np.argmax(y_pred, axis=1)
        plot_confusion_matrix(labels, y_pred_classes, classes=['Dense', 'Medium Dense', 'Sparse', 'None'])
        plot_accuracy_and_loss(history)
        plot_pr_curve(labels, y_pred, class_names=['Dense', 'Medium Dense', 'Sparse', 'None'])
        plot_roc_curve(labels, y_pred, class_names=['Dense', 'Medium Dense', 'Sparse', 'None'])

    print('Tüm eğitim ve değerlendirme süreçleri tamamlandı.')

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 150s 5s/step - accuracy: 0.1899 - loss: 14.2862 - val_accuracy: 0.2200 - val_loss: 61.6308
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 138s 6s/step - accuracy: 0.2688 - loss: 72.2319 - val_accuracy: 0.2200 - val_loss: 124.8530
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 121s 5s/step - accuracy: 0.2593 - loss: 134.5444 - val_accuracy: 0.2200 - val_loss: 188.3106
32/32 ━━━━━━━━━━━━━━━━━━━━ 116s 4s/step - accuracy: 0.5669 - loss: 112.9213
Epoch 1/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 131s 5s/step - accuracy: 0.2706 - loss: 10.0229 - val_accuracy: 0.2450 - val_loss: 47.8232
Epoch 2/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 156s 5s/step - accuracy: 0.2835 - loss: 59.5264 - val_accuracy: 0.2450 - val_loss: 105.0912
Epoch 3/3
25/25 ━━━━━━━━━━━━━━━━━━━━ 144s 5s/step - accuracy: 0.2702 - loss: 115.6207 - val_accuracy: 0.2450 - val_loss: 161.4873
32/32 ━━━━━━━━━━━━━━━━━━━━ 113s 4s/step - accuracy: 0.0495 - loss: 204.7993
Epoch 1/3
25/25 ━━━━━━━